In [14]:
import re
import pickle
import pandas as pd
import numpy as np

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

from gensim.models import Word2Vec

In [2]:
filename = "combine_df"
file = open(filename, 'rb')
tweets = pickle.load(file)
file.close()

In [3]:
tweets.head(5)

,id,label,tweet,tweet_token,tweet_token_filtered,tweet_stemmed,tweet_lemmatized
0,1,0.0,@user when a father is dysfunctional and is s...,"[when, father, is, dysfunctional, and, is, so,...","[father, dysfunctional, selfish, drags, kids, ...","[father, dysfunct, selfish, drag, kid, dysfunc...","[father, dysfunctional, selfish, drag, kid, dy..."
1,2,0.0,@user @user thanks for #lyft credit i can't us...,"[thanks, for, lyft, credit, can, not, use, cau...","[thanks, lyft, credit, use, cause, offer, whee...","[thank, lyft, credit, use, caus, offer, wheelc...","[thanks, lyft, credit, use, cause, offer, whee..."
2,3,0.0,bihday your majesty,"[bihday, your, majesty]","[bihday, majesty]","[bihday, majesti]","[bihday, majesty]"
3,4,0.0,#model i love u take with u all the time in ...,"[model, love, yoyou, take, with, yoyou, all, t...","[model, love, yoyou, take, yoyou, time, yoyour]","[model, love, yoyou, take, yoyou, time, yoyour]","[model, love, yoyou, take, yoyou, time, yoyour]"
4,5,0.0,factsguide: society now #motivation,"[factsguide, society, now, motivation]","[factsguide, society, motivation]","[factsguid, societi, motiv]","[factsguide, society, motivation]"


**1.** Создайте мешок слов с помощью sklearn.feature_extraction.text.CountVectorizer.fit_transform().<br>
Применим его к 'tweet_stemmed' и 'tweet_lemmatized' отдельно.

    ●	Игнорируем слова, частота которых в документе строго превышает порог 0.9 с помощью max_df.
    ●	Ограничим количество слов, попадающий в мешок, с помощью max_features = 1000.
    ●	Исключим стоп-слова с помощью stop_words='english'.
    ●	Отобразим Bag-of-Words модель как DataFrame. columns необходимо извлечь с помощью CountVectorizer.get_feature_names().

In [4]:
count_vectorizer = CountVectorizer(tokenizer=lambda doc: doc, max_df= 0.9, max_features = 1000, analyzer='word', binary=False, stop_words='english', lowercase=False)

# Создаем the Bag-of-Words модель
bag_of_stemmed_tweets = count_vectorizer.fit_transform(tweets['tweet_stemmed'])

# Отобразим Bag-of-Words модель как DataFrame
feature_names = count_vectorizer.get_feature_names()
pd.DataFrame(bag_of_stemmed_tweets.toarray(), columns = feature_names).head(5)

/opt/anaconda3/lib/python3.7/site-packages/sklearn/feature_extraction/text.py:386: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['b', 'c', 'd', 'e', 'f', 'g', 'h', 'k', 'l', 'm', 'n', 'o', 'p', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y'] not in stop_words.
  'stop_words.' % sorted(inconsistent))


,abl,absolut,accept,account,act,action,actor,actual,ad,adapt,...,yo,yoga,york,young,youtub,yoyou,yoyour,yoyoy,yr,yummi
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,2,1,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [5]:
bag_of_lemmatized_tweets = count_vectorizer.fit_transform(tweets['tweet_lemmatized'])
feature_names = count_vectorizer.get_feature_names()
pd.DataFrame(bag_of_lemmatized_tweets.toarray(), columns = feature_names).head(5)

,able,absolutely,account,act,action,actor,actually,adapt,add,adventure,...,yo,yoga,york,young,youtube,yoyou,yoyour,yoyoyou,yr,yummy
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,2,1,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


**2.** Создайте мешок слов с помощью sklearn.feature_extraction.text.TfidfVectorizer.fit_transform(). Применим его к 'tweet_stemmed' и 'tweet_lemmatized' отдельно.

    ●	Игнорируем слова, частота которых в документе строго превышает порог 0.9 с помощью max_df.
    ●	Ограничим количество слов, попадающий в мешок, с помощью max_features = 1000.
    ●	Исключим стоп-слова с помощью stop_words='english'.
    ●	Отобразим Bag-of-Words модель как DataFrame. columns необходимо извлечь с помощью TfidfVectorizer.get_feature_names().

In [6]:
tfidf_vectorizer = TfidfVectorizer(tokenizer=lambda doc: doc, max_df = 0.9, max_features = 1000, stop_words='english', lowercase=False)
values = tfidf_vectorizer.fit_transform(tweets['tweet_stemmed'])

# Show the Model as a pandas DataFrame
feature_names = tfidf_vectorizer.get_feature_names()
pd.DataFrame(values.toarray(), columns = feature_names).head(5)

/opt/anaconda3/lib/python3.7/site-packages/sklearn/feature_extraction/text.py:386: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['b', 'c', 'd', 'e', 'f', 'g', 'h', 'k', 'l', 'm', 'n', 'o', 'p', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y'] not in stop_words.
  'stop_words.' % sorted(inconsistent))


,abl,absolut,accept,account,act,action,actor,actual,ad,adapt,...,yo,yoga,york,young,youtub,yoyou,yoyour,yoyoy,yr,yummi
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.744148,0.392399,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0


In [7]:
values = tfidf_vectorizer.fit_transform(tweets['tweet_lemmatized'])

# Show the Model as a pandas DataFrame
feature_names = tfidf_vectorizer.get_feature_names()
pd.DataFrame(values.toarray(), columns = feature_names).head(5)

,able,absolutely,account,act,action,actor,actually,adapt,add,adventure,...,yo,yoga,york,young,youtube,yoyou,yoyour,yoyoyou,yr,yummy
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.00000,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.00000,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.00000,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.742113,0.39173,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.00000,0.0,0.0,0.0


**3.** Натренируем gensim.models.Word2Vec модель на наших данных.

    ●	Тренировать будем на токенизированных твитах combine_df['tweet_token']
    ●	Установим следующие параметры: size=200, window=5, min_count=2, sg = 1, hs = 0, negative = 10, workers= 32, seed = 34.
    ●	Используем функцию train() с параметром total_examples равным длине combine_df['tweet_token'], количество epochs установим 20.

In [8]:
modelW2V = Word2Vec(sentences=tweets['tweet_token'], size=200, window=5, min_count=2, sg = 1, hs = 0,
                    negative = 10, workers= 32, seed = 34)

modelW2V.train(tweets['tweet_token'], total_examples=modelW2V.corpus_count,  epochs=20)

(9149108, 11710940)

**4.** Давайте немного потестируем нашу модель Word2Vec и посмотрим, как она работает. Мы зададим слово positive = "dinner", и модель вытащит из корпуса наиболее похожие слова c помощью функции most_similar. То же самое попробуем со словом "trump".

In [9]:
modelW2V.wv.most_similar("dinner")

[('bolognese', 0.5736991763114929),
 ('spaghetti', 0.5584632158279419),
 ('bihdaydinner', 0.5493656396865845),
 ('saturdate', 0.5470651388168335),
 ('tacotuesday', 0.5427190065383911),
 ('cookout', 0.5407047271728516),
 ('whoopppp', 0.5289428234100342),
 ('shawarma', 0.5248156785964966),
 ('enroute', 0.524766206741333),
 ('starbuck', 0.523456871509552)]

In [10]:
modelW2V.wv.most_similar("trump")

[('donaldtrump', 0.5482732057571411),
 ('conman', 0.5347037315368652),
 ('dumptrump', 0.5328894853591919),
 ('unfavorability', 0.532060444355011),
 ('suppoer', 0.5270867943763733),
 ('donald', 0.5233590006828308),
 ('unfit', 0.5159579515457153),
 ('fuhered', 0.5158166885375977),
 ('trumptrain', 0.5107465386390686),
 ('phony', 0.5060904622077942)]

**5.** Из приведенных выше примеров мы видим, что наша модель word2vec хорошо справляется с поиском наиболее похожих слов для данного слова. Но как она это делает? Она изучила векторы для каждого уникального слова наших данных и использует косинусное сходство, чтобы найти наиболее похожие векторы (слова).
Давайте проверим векторное представление любого слова из нашего корпуса, например "food".

In [11]:
modelW2V['food']

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  """Entry point for launching an IPython kernel.


array([-1.67745277e-01, -1.06509931e-01, -6.18012309e-01,  2.93995649e-01,
        7.87495151e-02, -1.82881668e-01, -8.32565427e-01, -3.32734168e-01,
       -5.77480853e-01,  3.13773513e-01,  3.79191935e-01,  5.80631316e-01,
       -5.33308983e-01,  1.39798932e-02,  4.08772938e-02, -6.92352951e-01,
       -2.94036716e-01, -3.77851576e-01,  2.76067883e-01,  5.49632497e-02,
       -5.90843499e-01,  9.42766145e-02,  2.91568011e-01,  1.92352361e-03,
       -4.17825580e-02, -1.12588905e-01, -3.17453533e-01,  2.68818468e-01,
        5.99809736e-02,  2.02905059e-01, -1.78708464e-01, -4.31934237e-01,
       -6.87175840e-02,  2.55537421e-01,  9.82496589e-02,  6.47098184e-01,
       -1.65344626e-01,  1.39748320e-01,  1.24755248e-01, -2.25041404e-01,
        6.12658441e-01, -1.72521412e-01, -2.25947872e-01,  1.34153351e-01,
       -6.96196616e-01, -5.14550656e-02,  6.03293926e-02,  2.54581706e-03,
       -1.01952076e+00,  6.18514359e-01,  2.04247481e-04, -8.89482796e-02,
        1.20811857e-01,  

**6.** Поскольку наши данные содержат твиты, а не только слова, нам придется придумать способ использовать векторы слов из модели word2vec для создания векторного представления всего твита. Существует простое решение этой проблемы, мы можем просто взять среднее значение всех векторов слов, присутствующих в твите. Длина результирующего вектора будет одинаковой, то есть 200. Мы повторим тот же процесс для всех твитов в наших данных и получим их векторы. Теперь у нас есть 200 функций word2vec для наших данных.
Необходимо создать вектор для каждого твита, взяв среднее значение векторов слов, присутствующих в твите. В цикле сделать: vec += model_w2v[word].reshape((1, size))
и поделить финальный вектор на количество слов в твите.
На выходе должен получиться wordvec_df.shape = (49159, 200).


In [21]:
for tweet in tweets['tweet_token']:
    counter = 0
    n = 0
    for word in tweet:
        vector = np.zeros(200)
        if word in modelW2V:
            vector += modelW2V[word]
            n += 1
            
    if n > 0:
        vector = vector / n
        
    
            
    counter +=1
    if counter > 100000:
        break

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:6: DeprecationWarning: Call to deprecated `__contains__` (Method will be removed in 4.0.0, use self.wv.__contains__() instead).
  
/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:7: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  import sys
